# TradingView Crypto Screener

Dieses Jupyter Notebook fragt eine vorgegebene Liste von Handelspaaren von einer Börse (im Block *Settings* einstellen) ab, und erstellt eine Liste mit den lohnensten Paaren, um einen neuen Bot zu erstellen.

## Idee
TradingView stellt im Screener mehrere Handelssignale zur Verfügung. Hier wird das *Technical Rating* (TR) verwendet.

Was man in der normalen Screener-Ansicht nicht sehen kann: das Signal wird für verschiedene Intervalle angeboten. Es werden alle relevanten Werte von TradingView abgefragt. Das verwendete Signal berechnet sich wie folgt:

 **Bedingung**

 Vola (1D) > 1% `Volatility|1D > 1`

 & TR (1D): *Strong Buy* `Recommended.All > 0.5`
 
 & TR (1W): *Strong Buy* `Recommended.All|1W > 0.5`
 
 & TR (1M): *Buy* `Recommended.All|1M > 0.1`

 **Signal:**

 `MySignal = Volatility|1D * Recommended.All * Recommended.All|1W * Recommended.All|1M > 0.1`

 ## Anleitung

 vor dem ersten Ausführen des Codes:
 
 eine neue virtuelle Umgebung erzeugen (optional, aber zu empfehlen):

 ```
 virtualenv env
 source env/bin/activate
 ```

und dann die nötigen Libraries installieren:

 `pip install -r requirements.txt`


In [5]:
# Settings
exchange = 'BINANCE'

strategies = ['maufer', 'eazybot']

symbol_lists = {}
for strategy in strategies:
    with open(f'symbols_{strategy}.txt', 'r') as file:
        symbol_lists[strategy] = file.read().splitlines()

In [6]:
# Initialize Screener
import requests
import pandas as pd

url = "https://scanner.tradingview.com/crypto/scan"
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.4.1 Safari/605.1.15'
}

def create_payload(exchange, symbols):
    print(f"Exchange: {exchange.upper()}")
    symbols_with_exchange = [f"{exchange.upper()}:" + symbol for symbol in symbols] if symbols else []
    print(f"{len(symbols_with_exchange)} Symbols")
    filter_condition = [
            {"left": "exchange", "operation": "equal", "right": f"{exchange.upper()}"},
            # {"left": "average_volume_10d_calc", "operation": "egreater", "right": 1000000},
            {"left": "active_symbol", "operation": "equal", "right": True},
            {"left": "currency", "operation": "equal", "right": "USDT"}
        ]
    columns = [
            "name", "close", "Volatility.D", "Volatility.W",
            "24h_vol_change|5",
            "Recommend.Other|15", "Recommend.Other|60", "Recommend.Other", "Recommend.Other|1W", "Recommend.Other|1M",
            "Recommend.All|15", "Recommend.All|60", "Recommend.All", "Recommend.All|1W", "Recommend.All|1M",
            "change|1", "change|5", "change|15", "change|240", "change|60", "change", "change|1W", "change|1M"
        ]
    payload = {
        "filter": filter_condition,
        "options": {"lang": "en"},
        "filter2": {
            "operator": "and",
            "operands": [
                {
                    "operation": {
                        "operator": "or",
                        "operands": [
                            {
                                "expression": {"left": "type", "operation": "in_range", "right": ["spot"]}
                            }
                        ]
                    }
                }
            ]
        },
        "markets": ["crypto"],
        "symbols": {
            "query": {"types": []},
            "tickers": symbols_with_exchange
        },
        "columns": columns,
        "sort": {"sortBy": "Volatility.D", "sortOrder": "desc"},
        "price_conversion": {"to_symbol": False},
        "range": [0, 150]
    }

    return payload


In [7]:
# Get Data
from IPython.display import display

def get_data(url, headers, payload):
   data = None
   response = requests.post(url, headers=headers, json=payload)
   if response.status_code == 200:
       data = [[d["s"]] + d["d"] for d in response.json()['data']]
       cols= ["Symbol"] + payload['columns']
       df = pd.DataFrame(data,columns=cols)
   else:
       print(f"Web request returned an error: {response}")
   
   if data:
       df = pd.DataFrame(data,columns=cols)
       df['MySignal'] = df['Volatility.D'] * df['Recommend.All'] * df['Recommend.All|1W'] * df['Recommend.All|1W'] * df['Recommend.All|1M']
       positive_condition = ((df['Recommend.All|60'] > 0)
                                & (df['Recommend.All'] > 0.5)
                                & (df['Recommend.All|1W'] > 0.5)
                                & (df['Recommend.All|1M'] > 0.1)
                            )
       backup_condition = (df['Recommend.All'] > 0)  & (df['Recommend.All|1W'] > 0) & (df['Recommend.All|1M'] > 0)
       df['Vola'] = df['Volatility.D'].apply(lambda x: '{:.2f} %'.format(x))
       intervals = {'H': '|60', 'D': '', 'W': '|1W', 'M': '|1M'}
       for key, val in intervals.items():
          # df['TS_{key}'] = 'Strong Buy'
          df.loc[df[f'Recommend.All{val}'] > 0.5, f'Signal All 1{key}'] = 'Strong Buy'
          df.loc[df[f'Recommend.All{val}'] <= 0.5, f'Signal All 1{key}'] = 'Buy'
          df.loc[df[f'Recommend.All{val}'] <= 0.1, f'Signal All 1{key}'] = 'Neutral'
          df.loc[df[f'Recommend.All{val}'] < -0.1, f'Signal All 1{key}'] = 'Sell'
          df.loc[df[f'Recommend.All{val}'] < -0.5, f'Signal All 1{key}'] = 'Strong Sell'
          df.loc[df[f'Recommend.Other{val}'] > 0.5, f'Signal Other 1{key}'] = 'Strong Buy'
          df.loc[df[f'Recommend.Other{val}'] <= 0.5, f'Signal Other 1{key}'] = 'Buy'
          df.loc[df[f'Recommend.Other{val}'] <= 0.1, f'Signal Other 1{key}'] = 'Neutral'
          df.loc[df[f'Recommend.Other{val}'] < -0.1, f'Signal Other 1{key}'] = 'Sell'
          df.loc[df[f'Recommend.Other{val}'] < -0.5, f'Signal Other 1{key}'] = 'Strong Sell'
      
       df.rename(columns={'name': 'Name'}, inplace=True)
       df = df.sort_values(by='MySignal', ascending=False)
       df_filtered = df.loc[positive_condition]
       if not df_filtered.empty:
            display(df_filtered[['Name', 'Vola', 'MySignal', 'Signal All 1H', 'Signal All 1D', 'Signal All 1W', 'Signal All 1M', 'Signal Other 1H', 'Signal Other 1D', 'Signal Other 1W', 'Signal Other 1M']].reset_index(drop=True).head())
          # display(df_filtered[['Name', 'Vola', 'MySignal', 'Signal 1H', 'Signal 1D', 'Signal 1W', 'Signal 1M']].head())
       else:
           df_backup = df.loc[backup_condition]
           if not df_backup.empty:
               print(f"No matching symbols found - the best shots are:")
               display(df_backup[['Name', 'Vola', 'MySignal', 'Signal All 1H', 'Signal All 1D', 'Signal All 1W', 'Signal All 1M', 'Signal Other 1H', 'Signal Other 1D', 'Signal Other 1W', 'Signal Other 1M']].reset_index(drop=True).head())
           else:
               print(f"No matching symbols found!")
   

In [8]:
for symbol_list in symbol_lists.values():
    print(f"{len(symbol_list)} symbols")
    payload = create_payload(exchange, symbol_list)
    get_data(url, headers, payload)

221 symbols
Exchange: BINANCE
221 Symbols


,Name,Vola,MySignal,Signal All 1H,Signal All 1D,Signal All 1W,Signal All 1M,Signal Other 1H,Signal Other 1D,Signal Other 1W,Signal Other 1M
0,JASMYUSDT,11.22 %,1.091519,Buy,Strong Buy,Strong Buy,Strong Buy,Neutral,Buy,Buy,Buy
1,HIGHUSDT,12.33 %,1.048936,Buy,Strong Buy,Strong Buy,Buy,Neutral,Buy,Buy,Neutral
2,ARKMUSDT,3.79 %,0.418200,Buy,Strong Buy,Strong Buy,Strong Buy,Buy,Buy,Buy,Strong Buy


29 symbols
Exchange: BINANCE
29 Symbols
No matching symbols found - the best shots are:


,Name,Vola,MySignal,Signal All 1H,Signal All 1D,Signal All 1W,Signal All 1M,Signal Other 1H,Signal Other 1D,Signal Other 1W,Signal Other 1M
0,BCHUSDT,5.77 %,0.233366,Buy,Strong Buy,Buy,Buy,Buy,Buy,Neutral,Neutral
1,TWTUSDT,2.64 %,0.074778,Buy,Buy,Buy,Buy,Buy,Sell,Neutral,Buy
2,UNIUSDT,2.59 %,0.057481,Sell,Buy,Buy,Buy,Neutral,Sell,Neutral,Buy
3,BNBUSDT,2.11 %,0.054170,Neutral,Buy,Strong Buy,Buy,Neutral,Sell,Buy,Neutral
4,SOLUSDT,2.10 %,0.048374,Sell,Buy,Buy,Buy,Neutral,Neutral,Sell,Buy
